In [2]:
import os
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras import models, layers
import matplotlib.pyplot as plt

Train Model

In [7]:
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 15,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = [0.8, 1.2],
    horizontal_flip = True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(
    rescale= 1./255
)

train_generator = train_datagen.flow_from_directory(
    '../dataset/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

val_generator = val_test_datagen.flow_from_directory(
    '../dataset/val',
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'sparse'
)

test_generator = val_test_datagen.flow_from_directory(
    '../dataset/test',
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'sparse',
    shuffle = False
)



Found 2623 images belonging to 3 classes.
Found 561 images belonging to 3 classes.
Found 565 images belonging to 3 classes.


In [8]:
model = models.Sequential([
    layers.Input(shape=(224, 224,3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

In [9]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [11]:
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 15
)

Epoch 1/15
82/82 [==============================] - 76s 915ms/step - loss: 0.8075 - accuracy: 0.5909 - val_loss: 0.7991 - val_accuracy: 0.6150
Epoch 2/15
82/82 [==============================] - 68s 825ms/step - loss: 0.6923 - accuracy: 0.6740 - val_loss: 0.7275 - val_accuracy: 0.6756
Epoch 3/15
82/82 [==============================] - 70s 853ms/step - loss: 0.6439 - accuracy: 0.7053 - val_loss: 0.6670 - val_accuracy: 0.7112
Epoch 4/15
82/82 [==============================] - 67s 822ms/step - loss: 0.6040 - accuracy: 0.7507 - val_loss: 0.6653 - val_accuracy: 0.7362
Epoch 5/15
82/82 [==============================] - 64s 783ms/step - loss: 0.5856 - accuracy: 0.7560 - val_loss: 0.7133 - val_accuracy: 0.7166
Epoch 6/15
82/82 [==============================] - 66s 805ms/step - loss: 0.5469 - accuracy: 0.7835 - val_loss: 0.5586 - val_accuracy: 0.7861
Epoch 7/15
82/82 [==============================] - 67s 815ms/step - loss: 0.5362 - accuracy: 0.7960 - val_loss: 0.5802 - val_accuracy: 0.7683

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

18/18 [==============================] - 4s 223ms/step - loss: 0.5220 - accuracy: 0.7912
Test Loss: 0.5219672918319702
Test Accuracy: 0.7911504507064819


In [15]:
model.save('../models/coin_classifier.h5')